# Chapter 4: Storing Information

![](images/bobbytables.png)

Credit: [xkcd](https://xkcd.com/327/)

> *TL;DR: Data can be stored in different ways. Understanding different types of databases and the underlying data models is crucial for both storing and reading data.*

## Information (retrieval) models vs database models

### Information models

Formal discussions of information might start with the discussion of **information models**. [Wikipedia's definition](https://en.wikipedia.org/wiki/Information_model) reads:

> An information model in software engineering is a representation of concepts and the relationships, constraints, rules, and operations to specify [data semantics](https://en.wikipedia.org/wiki/Semantic_data_model) for a chosen domain of discourse. Typically it specifies relations between kinds of things, but may also include relations with individual things. It can provide sharable, stable, and organized structure of information requirements or knowledge for the domain context.

> An information model provides formalism to the description of a problem domain without constraining how that description is mapped to an actual implementation in software. There may be many mappings of the information model. Such mappings are called data models (...)

### Information retrieval models

Information retrieval then goes a step further and also provides theoretical models for retrieving information, as, for instance, we read in [Modern Information Retrieval](https://isbnsearch.org/isbn/9780321416919):

> Modeling in IR is a complex process aimed at producing a ranking function, i.e., a function that assigns scores to documents with regard to a given query. This process consists of two main tasks: (a) the conception of a logical framework for representing documents and queries and (b) the definition of a ranking function that computes a rank for each document with regard to a given query.

> To build a model, we think first of representations of the documents and of the user information needs. In the case of a document, the representation might be a subset of all terms in the document, which can be generated by removing stopwords (such as articles and prepositions) from the text, for instance. In the case of a query the representation might be a superset of the query terms formed by the original query enriched with synonyms, for instance. Given these representations, we then concieve the framework in which they can be modeled.

### Database models

However, while such models are obviously very interesting for information science, the discussion is also highly theoretical and requires no small amount of mathematics (set theory, algebra, probability theory, ...). 

Instead, from our practical perspective it makes more sense to open up the discussion of information, by dealing with **database models**. Indeed, in most real-world applications of information science, you will most likely have to deal with specific databases that store information and not with the abstract information models that map the data to these databases. 


## Databases

[Wikipedia](https://en.wikipedia.org/wiki/Database) defines a database as a "an organized collection of data, generally stored and accessed electronically from a computer system". Such a broad definition allows for many different kinds of databases, ranging from a single line of text (e.g. `apples,oranges,grapes` is a database!) to complex database management systems (DBMS) that operate on large data structures.

The classification of databases is a topic for a course on its own. For now, it will suffice to say that the development of database technology can be divided into three eras based on data model or structure: 

1. navigational
2. relational
3. post-relational

### Navigational

[Wikipedia](https://en.wikipedia.org/wiki/Navigational_database) says

>A navigational database is a type of database in which records or objects are found primarily by following **references from other objects**. The term was popularized by the title of Charles Bachman's 1973 Turing Award paper, The Programmer as Navigator. This paper emphasized the fact that the new disk-based database systems allowed the programmer to choose arbitrary navigational routes following relationships from record to record, contrasting this with the constraints of earlier magnetic-tape and punched card systems where data access was strictly sequential.

(...)

>Although Bachman described the concept of navigation in abstract terms, the idea of navigational access came to be associated strongly with the procedural design of the CODASYL Data Manipulation Language. Writing in 1982, for example, Tsichritzis and Lochovsky state that "The notion of currency is central to the concept of navigation." By the **notion of currency**, they refer to the idea that a program maintains (explicitly or implicitly) a current position in any sequence of records that it is processing, and that operations such as `GET NEXT` and `GET PRIOR` retrieve records relative to this current position, while also changing the current position to the record that is retrieved.

Underpinning the navigational model is the idea of **data as a table**.

One of the basic formats of tabular data is CSV, which is easy to work with in Python, through the built-in library `csv` or with `pandas`.

In [8]:
import csv
import os

headers = ["identifier", "last_name", "first_name"]
rows = [
            ["1", "Deneire", "Tom"],
            ["2", "Doe", "Jane"]
        ]

FILENAME = "mycsvfile.csv"

# Writing to csv file
with open(FILENAME, 'w') as csvfile:
    # creating a csv writer object
    csvwriter = csv.writer(csvfile)
    # writing the headers
    csvwriter.writerow(headers)
    # writing the data rows
    csvwriter.writerows(rows)

# Reading from csv file
data = []
with open(FILENAME, 'r') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
    # extracting each data row one by one
    for row in csvreader:
        data.append(row)

# test
result = {"headers":
            {"test": data[0], "original": headers},
          "rows":
            {"test": data[1:], "original": rows}
          }
for item in ["headers", "rows"]:
    try:
        assert result[item]["test"] == result[item]["original"]
    except AssertionError:
        print("test=", result[item]["test"])
        print("original=", result[item]["original"])

os.remove(FILENAME)

test= [['1', 'Deneire', 'Tom'], ['2', 'Doe', 'Jane']]
original= [[1, 'Deneire', 'Tom'], [2, 'Doe', 'Jane']]


### Relational

[Wikipedia:Relational_model](https://en.wikipedia.org/wiki/Relational_model) and [Wikipedia:Database](https://en.wikipedia.org/wiki/Database) say:

>The relational model (...) is an approach to managing data using a structure and language consistent with first-order predicate logic, first described in 1969 by English computer scientist Edgar F. Codd, where all **data is represented in terms of tuples, grouped into relations**. (...) he described a new system for storing and working with large databases. Instead of records being stored in some sort of linked list of free-form records (...), Codd's idea was to organise the **data as a number of "tables"**, each table being used for a different type of entity. Each table would contain a fixed number of columns containing the attributes of the entity. One or more columns of each table were designated as a primary key by which the rows of the table could be uniquely identified; cross-references between tables always used these primary keys, rather than disk addresses, and queries would join tables based on these key relationships, using a set of operations based on the mathematical system of relational calculus (from which the model takes its name). Splitting the data into a set of normalized tables (or relations) aimed to ensure that each "fact" was only stored once, thus simplifying update operations. Virtual tables called views could present the data in different ways for different users, but views could not be directly updated.

>The purpose of the relational model is to provide a **declarative method** for specifying data and queries: users directly state what information the database contains and what information they want from it, and let the database management system software take care of describing data structures for storing the data and retrieval procedures for answering queries.

>Most relational databases use the **SQL** data definition and query language.

Simply said, a relational database is a **collection of tables** that share a common data element. Have a look at this simplified example of a library catalogue:

**Table 1: titles**
| LOI        | title                   | language |
| ---------- | ----------------------- | -------- |
| c:1        | The origin of species   | eng      |
| c:2        | History of Middle Earth | eng      |

**Table 2: authors**
| LOI        | name                    | function |
| ---------- | ----------------------- | -------  |
| c:1        | Darwin, Charles         | aut      |
| c:2        | Tolkien, J.R.R.         | aut      |
| c:2        | Tolkien, Christopher    | edt      |

**Table 3: subjects**
| LOI        | subject               |
| ---------- | --------------------- |
| c:1        | evolutionary biology  |
| c:1        | theology              |
| c:1        | history of science    |
| c:2        | fantasy               |
| c:2        | constructed languages |

Now imagine what would happen if we were to convert these three tables to one spreadsheet:

```
LOI, title, language, name1, name2, function, subject1, subject2, subject3
```

And imagine scaling this up: books might easily have five authors and ten subjects, and we have only a little bit of metadata here. What about imprints, editions, carries, holdings, and so on? The advantages of relational databases are clear: they are perfect for storing and querying large amounts of related information in a flexible, decoupled way.

### Post-relational

[Wikipedia](https://en.wikipedia.org/wiki/NoSQL) says:

>A NoSQL (originally referring to "non-SQL" or "non-relational") database provides a mechanism for storage and retrieval of data that is modeled in **means other than the tabular relations used in relational databases**. Such databases have existed since the late 1960s, but the name "NoSQL" was only coined in the early 21st century (...) NoSQL databases are increasingly used in big data and real-time web applications. NoSQL systems are also sometimes called **"not only SQL"** to emphasize that they may support SQL-like query languages or sit alongside SQL databases (...).

>The data structures used by NoSQL databases (e.g. key–value pair, wide column, graph, or document) are different from those used by default in relational databases, making some operations faster in NoSQL. The particular suitability of a given NoSQL database depends on the problem it must solve. Sometimes the data structures used by NoSQL databases are also viewed as **more flexible** than relational database tables.

Categories of post-relational databases include:

* **[document stores](https://en.wikipedia.org/wiki/Document-oriented_database)**, such as JSON or XML
* **[triple stores](https://en.wikipedia.org/wiki/Triplestore)**, such as RDF
* **[key-value stores](https://en.wikipedia.org/wiki/Key%E2%80%93value_database)**, such as the MUMPS database by YottaDB that we use for Brocade, the University of Antwerp's Library Management System

The important thing to realize here is that post-relational databases can express things that relational databases cannot, at least not as easily. With post-relational database we leave the concept of tabular data behind, in favour of **data as hierarchies** (e.g. JSON or XML) or **data as graphs** (e.g. RDF).

For instance, how would you translate this JSON (postrelational) to a spreadsheet (relational)? Note the asymmetrical position of the key `age` in particular...

```JSON
{
	"1": {
		"name": "Tom Deneire",
		"age": "41"
	},
	"2": {
		"name": "Gandalf",
		"bio": {
			"age": "unknown",
			"occupation": "wizard"
		}
	}
}
```

### Hierarchies

#### JSON

In case you are not familiar with [JSON](https://www.json.org/json-en.html), [w3schools](https://www.w3schools.com/js/js_json_intro.asp) is a good starting point. 

One of the main advantages of JSON is that it is easy to read for humans (easier than XML) and that it can usually be parsed pretty easily by programming languages. For instance, JSON syntax is quite similar to that of Python dictionaries, as the example below shows. However, take care: an [array](https://www.w3schools.com/JS/js_json_arrays.asp) can also be a valid JSON structure!

In [20]:
json_string = '''
{
	"name": "Deneire",
	"age": 40,
	"initials": ["T", "B"]
}
'''
print(json_string)
print(type(json_string))

python_dict = {}
python_dict["name"] = "Deneire"
python_dict["age"] = 40
python_dict["initials"] = ["T", "B"]
print(python_dict)
print(type(python_dict))


{
	"name": "Deneire",
	"age": 40,
	"initials": ["T", "B"]
}

<class 'str'>
{'name': 'Deneire', 'age': 40, 'initials': ['T', 'B']}
<class 'dict'>


In fact, Python allows you to access JSON just like a database, using the `json` library to either turn JSON into an object (often a dict) with the function `loads()` or to turn an object into JSON with `dumps()`:

In [4]:
import json
contacts = """
{
	"1": {
		"lastname": "Doe",
		"firstname": "John"
	},
	"2": {
		"lastname": "Doe",
		"firstname": "Jane"
	}
}
"""
# Turn JSON into dict() with loads()
contacts_dict = json.loads(contacts)
print(contacts_dict["2"]["lastname"])

# Turn dict() into JSON with dumps()
contacts_dict["2"]["lastname"] = "Eyre"
contacts = json.dumps(contacts_dict)
print(contacts)

Doe
{"1": {"lastname": "Doe", "firstname": "John"}, "2": {"lastname": "Eyre", "firstname": "Jane"}}


#### XML

[XML](https://www.w3schools.com/xml/) is something most of you are probably familiar with, as it is a recurring technology in digital text analysis. In fact, XML is ubiquitous in the information world. For instance, it is very actively used in the library world. Another example is invoicing; for instance, the Government of Flanders has been asking for XML e-invoices from its suppliers for all contracts since 2017.

XML is properly queried with [XPath](https://www.w3schools.com/xml/xpath_intro.asp) and [XQuery](https://www.w3schools.com/xml/xquery_intro.asp), but with Python it is often easier to use one of the available XML libraries which turn XML's hierarchical structure into a parse tree, which behaves like a Pythonic object that you can iterate over. Two well-known examples are 

- [beautifulsoup](https://pypi.org/project/beautifulsoup4/)
- [lxml](https://lxml.de/)

Here we will briefly discuss reading, parsing (the process of analyzing XML documents to extract their information) and building XML with `lxml`.

#### Install lxml

Depending on your OS and programming environment, you might first need to install lxml. You can easily verify this by executing `import lxml`. If the module is not present, Python will throw a `ModuleNotFoundError`.

For more instructions on this, visit the [lxml](https://lxml.de/installation.html) or [Anaconda](https://anaconda.org/anaconda/lxml) documentation.

#### Reading XML

In [6]:
import lxml.etree

xml_string = b'''
<database>
    <record nr="1">
        <name type="last">Deneire</name>
        <name type="first">Tom</name>
    </record>
    <record nr="2">
        <name type="last">Doe</name>
        <name type="first">John</name>
    </record>
</database>
'''
# Turn the XML bytes-string into an `etree` object
tree = lxml.etree.fromstring(xml_string, parser=None)
print(tree)
# Show the methods that this object allows
print(dir(tree))

<Element database at 0x7f888444e380>
['__bool__', '__class__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '_init', 'addnext', 'addprevious', 'append', 'attrib', 'base', 'clear', 'cssselect', 'extend', 'find', 'findall', 'findtext', 'get', 'getchildren', 'getiterator', 'getnext', 'getparent', 'getprevious', 'getroottree', 'index', 'insert', 'items', 'iter', 'iterancestors', 'iterchildren', 'iterdescendants', 'iterfind', 'itersiblings', 'itertext', 'keys', 'makeelement', 'nsmap', 'prefix', 'remove', 'replace', 'set', 'sourceline', 'tag', 'tail', 'text', 'values', 'xpath']


#### Parsing XML

In [2]:
# Iterate over the "record" elements
for element in tree.iter("record"):
    print(element)

<Element record at 0x7fb764770740>
<Element record at 0x7fb7647b3ac0>


In [9]:
# Iterate over the "name" elements
for element in tree.iter("name"):
    print(element)

<Element name at 0x7f887d14a880>
<Element name at 0x7f887d14b380>
<Element name at 0x7f887d14a4c0>
<Element name at 0x7f887d14aa40>


In [3]:
# Access the "name" elements
for element in tree.iter("name"):
    # XML attributes are dicts
    for attribute_name, attribute_value in element.items():
        print(attribute_name, ":", attribute_value)
        # Get text with .text method of element object
        print(element.text)

type : last
Deneire
type : first
Tom
type : last
Lipsius
type : first
Justus


#### Writing XML

In [7]:
# Constructing the aforementioned XML string

import lxml.etree

root = lxml.etree.Element("database")

# Create subelements with SubElement(parent, tag, attribute)
record1 = lxml.etree.SubElement(root, "record", nr="1")
name1 = lxml.etree.SubElement(record1, "name", type="last")
name1.text = "Deneire"
name2 = lxml.etree.SubElement(record1, "name", type="first")
name2.text = "Tom"

record2 = lxml.etree.SubElement(root, "record", nr="2")
name3 = lxml.etree.SubElement(record2, "name", type="last")
name3.text = "Doe"
name4 = lxml.etree.SubElement(record2, "name", type="first")
name4.text = "John"

print(lxml.etree.tostring(root))

b'<database><record nr="1"><name type="last">Deneire</name><name type="first">Tom</name></record><record nr="2"><name type="last">Doe</name><name type="first">John</name></record></database>'


### Graphs

#### RDF

to do: make this more explicit and introduce .nt type for triples (much easier than xml!)

Unfortunately, we do not have time to discuss RDF and Linked Data in detail. However, it is important to realize that RDF is a **data model, not a data serialization model**, such as XML or JSON - in fact, both can be used to express RDF data.

A quick summary from [Wikipedia](https://en.wikipedia.org/wiki/Resource_Description_Framework):

>The RDF data model is similar to classical conceptual modeling approaches (such as entity–relationship or class diagrams). It is based on the idea of making statements about resources (in particular web resources) in expressions of the form **subject–predicate–object, known as triples**. The subject denotes the resource, and the predicate denotes traits or aspects of the resource, and expresses a relationship between the subject and the object.
>
>For example, one way to represent the notion "The sky has the color blue" in RDF is as the triple: a **subject** denoting "the sky", a **predicate** denoting "has the color", and an **object** denoting "blue". 
>
>RDF is an abstract model with several serialization formats (i.e. file formats), so the particular encoding for resources or triples varies from format to format.

In order to define the data used in RDF we use a **Uniform Resource Identifier (URI)**, a unique and unambiguous identifier for all things described in the triplestore. To illustrate how this works, let's look at the part of the RDF/XML for the Wikidata entry [Paris](https://www.wikidata.org/wiki/Q90), known as entity `Q90`:

```xml
<?xml version="1.0"?>
    <rdf:Description rdf:about="http://www.wikidata.org/entity/Q90">
        <wdt:P1376 rdf:resource="http://www.wikidata.org/entity/Q142"/>
    </rdf:Description>
```

This is a triple expressing the fact that:

1. "Paris" (subject) 
2. "is the capital of" (predicate) 
3. "France" (object)

To express this three URIs are used: 

1. `Q90` (Paris)
2. `P1376` (property "is capital of") 
3. `Q142` (France)

Theismakes the statement unique, uniform, unambiguous (telling a computer that Paris the city, not Paris Hilton (Q47899) is the capital of France), and also **Linked Data**: all elements of the triple are linked up to other data, e.g. `Q142` which was the object in this statement, will be the subject of others.

Consider the difference with non-RDF XML where resources are not identified with URIs and data is not linked and the model not open:

```xml
<?xml version="1.0"?>
    <Description>Paris
        <isCapitalof>France</isCapitalof>
    </Description>
```

RDF and especially Linked Open Data are undoubtedly part of the future for information science. If you're interested to know more or need to work with RDF and Linked Data in practice, I highly recommend reading *A Librarian's Guide to Graphs, Data and the Semantic Web*, By James Powell, [ISBN 978-1843347538](https://isbnsearch.org/isbn/9781843347538).

### Data(bases) as Linked Data

The characterization of databases can also be considered not so much from a technical standpoint, but rather as a gradual evolution on the [Linked Data](https://www.w3.org/2011/gld/wiki/5_Star_Linked_Data) scale:

>Tim Berners-Lee, the inventor of the Web and initiator of the Linked Data project, suggested a 5 star deployment scheme for Linked Data. The 5 Star Linked Data system is cumulative. Each additional star presumes the data meets the criteria of the previous step(s).

>☆ Data is available on the Web, in whatever format.	

>☆☆ Available as machine-readable structured data, (e.g., not a scanned image).

>☆☆☆ Available in a non-proprietary format, (e.g, CSV, not Microsoft Excel).

>☆☆☆☆ Published using open standards from the W3C (RDF and SPARQL).

>☆☆☆☆☆ All of the above and links to other Linked Open Data.

In this way, we can organize different database types into a data hierarchy like so:

![](images/5-star-steps-open-data-5-star-model.png)

* OL: Open License
* RE: Readable
* OF: Open format
* URI: Uniform Resource Identifier
* LD: Linked Data

If you want to know more about this, [this article](https://www.ontotext.com/knowledgehub/fundamentals/five-star-linked-open-data/) offers a good explanation.